<b> Description: </b> In this notebook we generate knowledge statements about ROC stories.

## 1. Import libraries

In [2]:
import pandas as pd
import os
import json
import numpy as np
from tqdm import tqdm

In [40]:
import random

In [2]:
from gpt_generator import request

In [3]:
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("API_KEY")

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

## 2. Load datasets

Load the winter 2017 ROCstories json file with mixed labels as incorrect labels. 

In [12]:
df = pd.read_json("../dataset/ROCStories_winter2017_mixed.json")
df.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,ending0,ending1,label,story,choices,story_triple
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...","In the end, he passed with flying colours.",0,David noticed he had put on a lot of weight re...,"[After a few weeks, he started to feel much be...","[[David, noticed, lot weight], [He, examined, ..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,Frank is glad he exercises more now.,0,Tom had a very short temper. One day a guest m...,[Tom sat on his couch filled with regret about...,"[[Tom, had, temper], [guest, made, ], [He, pun..."
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,She threw up and felt better.,0,Marcus needed clothing for a business casual e...,[Marcus was happy to have the right clothes fo...,"[[Marcus, needed, clothing event], [All, were,..."
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,She was then able to print the documents with ...,0,Bobby thought Bill should buy a trailer and ha...,[He ended up buying the truck he wanted despit...,"[[Bobby, thought, trailer it car], [Bill, thou..."
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,I drove home disappointed that my night didn't...,His congregation was delighted and so was he.,1,John was a pastor with a very bad memory. He t...,[I drove home disappointed that my night didn'...,"[[John, was, memory], [He, tried, sermons adva..."


Read prompt for knowledge generations.

In [ ]:
def saveContents(arr: list, output_path:str):
    pd.DataFrame(arr)[["storyid","storytitle","sentence1","sentence2","sentence3","sentence4","ending0","ending1", "label", "story", "know"]].to_csv(output_path)
a
with open("../knowledge_prompts/knowledge/prompts/cloze_prompt.txt", "r") as f:
    context_placeholder = f.read()

def prompt_placeholder(context_string, new_question):
    return {'role':'user', 'content': context_string.replace('{question}', new_question)}

Split dataset into 5 batches to accomodate OPENAI's API max rate. 

In [ ]:
split_no = 5
for split_no, split_df in enumerate(np.array_split(df, split_no), start=1):
    split_df.to_json(f"../dataset/winter_2017_split_{split_no}.json")

Generate knowledege for batches.

In [ ]:
curr_df = pd.read_json("../dataset/winter_2017_split_1.json")

In [ ]:
generated_examples = []
output_path = "../dataset/split1.csv"

for _, row in tqdm(curr_df.iterrows(), total=curr_df.shape[0]):
    try:
        prompt_to_pass = prompt_placeholder(context_placeholder, row["story"])

        context_knowledge = request(
            prompt_to_pass,
            client=client)

        row['know'] = context_knowledge
        generated_examples.append(row.to_dict())
        

    except KeyboardInterrupt:
        print("Interrupted! Saving contents...")
        saveContents(generated_examples, output_path)


saveContents(generated_examples, output_path)

100%|██████████| 10533/10533 [1:55:46<00:00,  1.52it/s]  


In [ ]:
curr_df = pd.read_json("../dataset/winter_2017_split_2.json")

In [ ]:
generated_examples = []
output_path = "../dataset/split2.csv"

for _, row in tqdm(curr_df.iterrows(), total=curr_df.shape[0]):
    try:
        prompt_to_pass = prompt_placeholder(context_placeholder, row["story"])

        context_knowledge = request(
            prompt_to_pass,
            client=client)

        row['know'] = context_knowledge
        generated_examples.append(row.to_dict())
        

    except KeyboardInterrupt:
        print("Interrupted! Saving contents...")
        saveContents(generated_examples, output_path)


saveContents(generated_examples, output_path)

 96%|█████████▌| 10094/10533 [22:29:36<1:05:59,  9.02s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-N52bJZsDufSXzNr9a3BMIvBc on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying...


100%|██████████| 10533/10533 [23:32:47<00:00,  8.05s/it]  


In [ ]:
curr_df = pd.read_json("../dataset/winter_2017_split_3.json")

In [ ]:
generated_examples = []
output_path = "../dataset/split3.csv"

for _, row in tqdm(curr_df.iterrows(), total=curr_df.shape[0]):
    try:
        prompt_to_pass = prompt_placeholder(context_placeholder, row["story"])

        context_knowledge = request(
            prompt_to_pass,
            client=client)

        row['know'] = context_knowledge
        generated_examples.append(row.to_dict())
        

    except KeyboardInterrupt:
        print("Interrupted! Saving contents...")
        saveContents(generated_examples, output_path)


saveContents(generated_examples, output_path)

100%|██████████| 10533/10533 [25:16:39<00:00,  8.64s/it]  


In [ ]:
curr_df = pd.read_json("../dataset/winter_2017_split_4.json")

In [ ]:
generated_examples = []
output_path = "../dataset/split4.csv"

for _, row in tqdm(curr_df.iterrows(), total=curr_df.shape[0]):
    try:
        prompt_to_pass = prompt_placeholder(context_placeholder, row["story"])

        context_knowledge = request(
            prompt_to_pass,
            client=client)

        row['know'] = context_knowledge
        generated_examples.append(row.to_dict())
        

    except KeyboardInterrupt:
        print("Interrupted! Saving contents...")
        saveContents(generated_examples, output_path)


saveContents(generated_examples, output_path)

100%|██████████| 10533/10533 [25:16:55<00:00,  8.64s/it]  


In [ ]:
curr_df = pd.read_json("../dataset/winter_2017_split_5.json")

In [ ]:
generated_examples = []
output_path = "../dataset/split5.csv"

for _, row in tqdm(curr_df.iterrows(), total=curr_df.shape[0]):
    try:
        prompt_to_pass = prompt_placeholder(context_placeholder, row["story"])

        context_knowledge = request(
            prompt_to_pass,
            client=client)

        row['know'] = context_knowledge
        generated_examples.append(row.to_dict())
        

    except KeyboardInterrupt:
        print("Interrupted! Saving contents...")
        saveContents(generated_examples, output_path)


saveContents(generated_examples, output_path)

  4%|▎         | 394/10533 [07:45<3:02:38,  1.08s/it]

Error code: 503 - {'error': {'code': 503, 'message': 'Service Unavailable.', 'param': None, 'type': 'cf_service_unavailable'}}
Retrying...


  4%|▍         | 411/10533 [09:03<2:35:04,  1.09it/s] 

Error code: 503 - {'error': {'code': 503, 'message': 'Service Unavailable.', 'param': None, 'type': 'cf_service_unavailable'}}
Retrying...


  9%|▉         | 974/10533 [20:18<2:58:20,  1.12s/it] 

Error code: 503 - {'error': {'code': 503, 'message': 'Service Unavailable.', 'param': None, 'type': 'cf_service_unavailable'}}
Retrying...


 10%|▉         | 1030/10533 [22:29<2:25:01,  1.09it/s]

Error code: 503 - {'error': {'code': 503, 'message': 'Service Unavailable.', 'param': None, 'type': 'cf_service_unavailable'}}
Retrying...


100%|██████████| 10533/10533 [17:43:20<00:00,  6.06s/it]  


Concatenate all batches into one dataset and save.

In [ ]:
out_df = pd.DataFrame()
for num in ["1", "2", "3", "4", "5"]:
    temp = pd.read_csv(f"../dataset/split{num}.csv", index_col=0)
    out_df = pd.concat([out_df, temp], ignore_index=True)

out_df.shape

(52665, 11)

In [ ]:
out_df.to_json("../dataset/all_know.json")

In [ ]:
out_df[out_df.columns[:-1]].to_json("../dataset/all.json")

## 3. Combine and Export

Load 2017 dataset with generated gpt-3.5 endings and format it.

In [49]:
labeled_df = pd.DataFrame()

def shuffle_list(x):
    random.shuffle(x)
    return x

for idx in range(1,6):
    ds_idx = pd.read_csv(f"../dataset/winter_2017_labeled_p{idx}.csv", index_col=0)
    ds_idx["cands"] = ds_idx.apply(lambda x: [x["correctE"], x["incorrectE"]], axis=1)
    ds_idx["cands"] = ds_idx.apply(lambda x: shuffle_list(x["cands"]), axis=1)
    ds_idx["story"] = ds_idx.apply(lambda x: " ".join([x[f"sentence{y}"] for y in range(1, 5)]), axis=1)
    ds_idx.rename(columns={"correctE":"answer"}, inplace=True)
    del ds_idx["incorrectE"]
 
    labeled_df = pd.concat([labeled_df, ds_idx], ignore_index=True)

print("Data shape:", labeled_df.shape)
labeled_df.sample(2)

Data shape: (52664, 9)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,answer,cands,story
9706,ca794399-5823-4b5a-b5d7-5b7357cab868,No longer skinny,Allan was a very skinny man.,He hated the way he looked.,One day he decided that he was going to put on...,He began to drink protein shakes and eat 10 ti...,Allan is no longer very skinny.,"[Allan is no longer very skinny., Allan became...",Allan was a very skinny man. He hated the way ...
27707,cd329f74-eb31-4cd6-961b-b584f05e3534,Veterinarian visit,Dillon needed his cat to get its annual vaccin...,He called the veterinarian office and made an ...,He used some cat treats to lure his cat into t...,Dillon walked with the cat carrier to the vete...,The veterinarian gave the cat the required vac...,[The veterinarian gave the cat the required va...,Dillon needed his cat to get its annual vaccin...


Add the knowledge statements to the dataset.

In [52]:
know_df = pd.DataFrame()
for idx in range(1,6):
    ds_idx = pd.read_csv(f"../dataset/split{idx}.csv", index_col=0)
    know_df = pd.concat([know_df, ds_idx], ignore_index=True)
know_df.shape

(52665, 11)

In [58]:
joined_df = pd.merge(labeled_df, know_df[["storyid", "know"]], on="storyid", how="inner")
joined_df.sample(2)

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,answer,cands,story,know
10474,727e8ca5-6831-4912-933c-f2ed59ceda0e,Nina gets a new Blender,Nina decided that she wanted a smoothie.,She put all of the ingredients into her blender.,She turned the blender on but nothing happened.,Nina decided she needed a new blender.,She went to the store and bought a new one.,"[She went to the store and bought a new one., ...",Nina decided that she wanted a smoothie. She p...,"Blenders are kitchen appliances used to mix, p..."
34383,0e250fc6-8c79-4f50-93b4-2efc4dc946c4,Olympics,My mom wanted to watch the Olympics with me re...,She told me what channel they were on and I sw...,During the Olympics we cheered for our country!,We both really enjoyed it so we watched it the...,We ended up watching the Olympics every day th...,[We ended up watching the Olympics every day t...,My mom wanted to watch the Olympics with me re...,The Olympics is a multi-sport event held every...


In [59]:
joined_df.shape

(52664, 10)

In [60]:
joined_df.to_json("../dataset/2017_gpt_data.json")